In [65]:
import fiona
from fiona.crs import to_string
import pprint
import shapely
from shapely.geometry import Polygon
import geopandas as gpd
from rasterstats import zonal_stats
import pandas as pd

In [30]:
c = fiona.open('data/geo/countries/countries.shp')
'driver: %s' % c.driver.encode('latin-1'), 'crs: %s' % to_string(c.crs), 'records: %s' % len(c), c.bounds

('driver: ESRI Shapefile',
 'crs: +init=epsg:4326',
 'records: 255',
 (-179.9999999999999, -90.0, 180.0000000000002, 83.63410065300012))

In [31]:
rec = next(c)
set(rec.keys()) - set(c.schema.keys())
pprint.pprint(c.schema)

{'geometry': 'Polygon',
 'properties': OrderedDict([(u'scalerank', 'int:4'), (u'featurecla', 'str:30'), (u'LABELRANK', 'float:6.2'), (u'SOVEREIGNT', 'str:32'), (u'SOV_A3', 'str:3'), (u'ADM0_DIF', 'float:4.2'), (u'LEVEL', 'float:4.2'), (u'TYPE', 'str:17'), (u'ADMIN', 'str:40'), (u'ADM0_A3', 'str:3'), (u'GEOU_DIF', 'float:4.2'), (u'GEOUNIT', 'str:40'), (u'GU_A3', 'str:3'), (u'SU_DIF', 'float:4.2'), (u'SUBUNIT', 'str:40'), (u'SU_A3', 'str:3'), (u'BRK_DIFF', 'float:4.2'), (u'NAME', 'str:36'), (u'NAME_LONG', 'str:40'), (u'BRK_A3', 'str:3'), (u'BRK_NAME', 'str:36'), (u'BRK_GROUP', 'str:30'), (u'ABBREV', 'str:13'), (u'POSTAL', 'str:4'), (u'FORMAL_EN', 'str:52'), (u'FORMAL_FR', 'str:35'), (u'NOTE_ADM0', 'str:22'), (u'NOTE_BRK', 'str:164'), (u'NAME_SORT', 'str:36'), (u'NAME_ALT', 'str:38'), (u'MAPCOLOR7', 'float:4.2'), (u'MAPCOLOR8', 'float:4.2'), (u'MAPCOLOR9', 'float:4.2'), (u'MAPCOLOR13', 'float:6.2'), (u'POP_EST', 'float:13.2'), (u'GDP_MD_EST', 'float:11.2'), (u'POP_YEAR', 'float:7.2'), (u'

In [64]:
shp = 'data/geo/countries/countries_nf2.shp'
gdf = gpd.GeoDataFrame.from_file(shp); gdf.head(3)

,ADMIN,CONTINENT,ISO_A3,REGION_UN,REGION_WB,SUBREGION,WB_A3,geometry
0,Aruba,North America,ABW,Americas,Latin America & Caribbean,Caribbean,ABW,POLYGON ((-69.99693762899992 12.57758209800004...
1,Afghanistan,Asia,AFG,Asia,South Asia,Southern Asia,AFG,"POLYGON ((71.04980228700009 38.40866445000009,..."
2,Angola,Africa,AGO,Africa,Sub-Saharan Africa,Middle Africa,AGO,(POLYGON ((11.73751945100014 -16.6925779829998...


In [59]:
raster = 'data/geo/images/F142002.v4b_web.stable_lights.avg_vis.tif'
shp = 'data/geo/countries/countries_nf2.shp'
stats = zonal_stats(shp, raster)
stats[1].keys()

/Users/peter/anaconda/lib/python2.7/site-packages/rasterstats/io.py:292: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/peter/anaconda/lib/python2.7/site-packages/rasterstats/io.py:299: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  new_array = self.src.read(self.band, window=win, boundless=True, masked=masked)


['count', 'max', 'mean', 'min']

In [74]:
li_stats = []
for i in range(len(stats)):
    li_stats.append(stats[i]['mean'])
li_stats[:3]

[34.333333333333336, 0.03426151517975618, 0.03442736150843421]

In [78]:
s_stats = pd.Series(li_stats, name='light'); s_stats.head(3)

0    34.333333
1     0.034262
2     0.034427
Name: light, dtype: float64

In [82]:
gdf = gdf.join(s_stats); gdf.head(3)

,ADMIN,CONTINENT,ISO_A3,REGION_UN,REGION_WB,SUBREGION,WB_A3,geometry,light
0,Aruba,North America,ABW,Americas,Latin America & Caribbean,Caribbean,ABW,POLYGON ((-69.99693762899992 12.57758209800004...,34.333333
1,Afghanistan,Asia,AFG,Asia,South Asia,Southern Asia,AFG,"POLYGON ((71.04980228700009 38.40866445000009,...",0.034262
2,Angola,Africa,AGO,Africa,Sub-Saharan Africa,Middle Africa,AGO,(POLYGON ((11.73751945100014 -16.6925779829998...,0.034427
